In [96]:
%matplotlib qt
import math
from matplotlib import pyplot as plt
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
from pyxem.utils.sim_utils import sim_as_signal

import os
import addcopyfighandler

from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

from transforms3d.euler import axangle2euler
import diffpy.structure

# Indexing from peaks 2D

Notebook to index diffraction peaks from 2D detector images, in order to find the zone axis of the diffraction pattern from simulations of diffraction crystal files.

### Set all the variables first:

In [97]:
### VARIABLE PARAMETERS ###

# PEAK VECTORS
## Paste the peaks coordinates found in the `find_peaks` function
peaks2D = np.array(
[[0.03505, 0.02103],
 [-0.13319, 0.21731],
 [-0.03505, -0.27338999999999997],
 [-0.38555, 0.10515],
 [0.45565, -0.049069999999999996],
 [0.41358999999999996, -0.20329],
 [-0.30143, 0.39957],
 [0.62389, -0.23132999999999998],
 [-0.44162999999999997, 0.73605]]
)

## Select which two peaks to use (provide the index)
p1_index = 1
p2_index = 4
## Include the error in length and in angle (error_angle * 2) you will permit.
error_angle = 4
error_length = 0.1
dp_is_centered = True

# SIMULATION DETECTOR PARAMETERS:
pattern_size = 256  # pixels
reciprocal_radius = 1.5
beam_energy = 300.0
max_excitation_error = 0.025

# EXTERNAL FILES:
## Select structure file name
structure_name = 'FAPbI3'
## The crossproduct dhkl file
path_dhkl = os.path.relpath('data/structures/FAPbI3_Cubic_ReflexionList.txt')
crystal_structure = 'cubic'
## The CIF files to use to simulate diffraction
load_structure_path = os.path.relpath('data/structures/FAPbI3_Cubic.cif')

## Calculate the length and vector angle of the selected peaks

In [98]:
v0 = (0.,0.)
v1 = peaks2D[p1_index]
v2 = peaks2D[p2_index]

#Calculate vector length and vector angle
length_1 = np.linalg.norm(v1-v0)
length_2 = np.linalg.norm(v2-v0)
error_1 = length_1*0.1
error_2 = length_2*0.1

angle_12 = np.arccos((np.dot(v1-v0, v2-v0))/(length_1 * length_2))
angle_12 = np.rad2deg(angle_12)

print(length_1, length_2, angle_12)

0.2548788186570238 0.4582846139682195 127.65089237885087


## Define useful functions

### Hexagonal structure
Determines the interplanar angle for two planes in a hexagonal structure:
- `v1, v2` are the planes of interest.
- `a,b,c` are the unit cell parameters.

See [equation 2-6](http://iucr.or.kr/pdf/18(1)07-09.pdf) for derivation.

In [99]:
def hex_inter_angle(v1, v2, a, b, c):
    h1=v1[0]
    k1=v1[1]
    l1=v1[2]
    h2=v2[0]
    k2=v2[1]
    l2=v2[2]
    z=((h1*h2 + k1*k2 +(h1*k2 +k1*h2)/2 + ((3*a**2)/(4*c**2))*l1*l2)
       /(math.sqrt((h1**2+k1**2+h1*k1+((3*a**2)/(4*c**2))*l1**2)*
                   (h2**2+k2**2+h2*k2+((3*a**2)/(4*c**2))*l2**2))))
    theta = math.acos(z)
    return np.rad2deg(theta)

### Cubic, tetragonal and Orthorhombic structure
Determines the interplanar angle for two planes in a cubic/orthorhombic or tetragonal structure:
- `v1, v2` are the planes of interest
- `a,b,c` are the unit cell parameters

See [equation 2-7](http://iucr.or.kr/pdf/18(1)07-09.pdf) for derivation.

In [100]:
def cto_inter_angle(v1, v2, a, b, c):
    h1=v1[0]
    k1=v1[1]
    l1=v1[2]
    h2=v2[0]
    k2=v2[1]
    l2=v2[2]
    z=(((h1*h2/a**2)+(k1*k2/b**2)+(l1*l2/c**2))
       /(((h1**2)/(a**2)+(k1**2)/(b**2)+(l1**2)
          /(c**2))**0.5*((h2**2)/(a**2)+(k2**2)/(b**2)+(l2**2)/(c**2))**0.5))
    theta = math.acos(z)
    return np.rad2deg(theta)


## Find possible orientations that match the experimental input vectors and angle

In [101]:
# Load dhkl file
data = np.loadtxt(path_dhkl, skiprows=9,usecols=(0,1,2,3))
data[:,3] = 1/data[:,3]
data

array([[ 1.        ,  0.        ,  0.        ,  0.15720057],
       [ 0.        ,  0.        ,  1.        ,  0.15720057],
       [ 0.        ,  1.        ,  0.        ,  0.15720057],
       ...,
       [ 3.        , -8.        ,  3.        ,  1.42350781],
       [-8.        ,  3.        ,  3.        ,  1.42350781],
       [ 3.        ,  3.        , -8.        ,  1.42350781]])

In [102]:
# First, get the unit cell lengths from the structure file and store as a, b and c:

unit_cell_len = {'a': None, 'b': None, 'c':None}
for row in data:
    if row[0] == 1. and row[1] == 0. and row[2] == 0.:
        unit_cell_len['a'] = row[3]
    if row[0] == 0. and row[1] == 1. and row[2] == 0.:
        unit_cell_len['b'] = row[3]
    if row[0] == 0. and row[1] == 0. and row[2] == 1.:
        unit_cell_len['c'] = row[3]
    if unit_cell_len['a'] is not None and unit_cell_len['b'] is not None and unit_cell_len['c'] is not None :
        break

a = 1/ unit_cell_len['a']
b = 1/ unit_cell_len['b']
c = 1/ unit_cell_len['c']
print(a,b,c)

6.3613 6.3613 6.3613


In [103]:
where_1 = np.where(np.logical_and(data[:,3]>(length_1-error_1), data[:,3]<(length_1+error_1)))
where_2 = np.where(np.logical_and(data[:,3]>(length_2-error_2), data[:,3]<(length_2+error_2)))
vectors_1 = data[where_1,0:3]
vectors_2 = data[where_2,0:3]
orientation = []
for i in range(0,len(vectors_1[0])):
    for k in range(0, len(vectors_2[0])):
        if crystal_structure == 'hexagonal':
            angle = hex_inter_angle(vectors_1[0,i], vectors_2[0,k], a, b, c)
        elif crystal_structure == 'cubic' or crystal_structure == 'cubic' or crystal_structure == 'cubic':
            angle = cto_inter_angle(vectors_1[0,i], vectors_2[0,k], a, b, c)
        else:
            raise ValueError("This notebook only supports cubic, tetragonal, orthorhombic and hexagonal lattice structures.")
        if abs(angle-angle_12) < error_angle:
            cross = np.cross(vectors_1[0,i], vectors_2[0,k])
            orientation.append(cross)

In [104]:
orientation = np.array(orientation)
len(orientation)

9

In [105]:
print("Vectors:")
print("----------------")
print(orientation)

Vectors:
----------------
[[ 3. -3.  0.]
 [ 3.  0.  3.]
 [ 0. -3. -3.]
 [ 0.  3. -3.]
 [-3.  0. -3.]
 [ 3.  3.  0.]
 [-3.  0.  3.]
 [ 0.  3.  3.]
 [-3. -3.  0.]]


In [106]:
#roi_letter = 'G'
#cif_name = 'FAPbI3'
#save_path = '../../../data/SED/roi/roi_{}/indexing/{}_{}_l1_{}_l2_{}_a12_{}'.format(roi_letter,cif_name, crystal_structure, length_1,length_2,angle_12)
#np.save(save_path, reduce_vectors)

# Simulating possible zone axis
From the possible vectors as zone axis

In [107]:
vectors = orientation
len(vectors)

9

## Load diffpy structure file (`.cif`)

In [108]:
cif_name = os.path.basename(load_structure_path).split('.')[0]

structure = diffpy.structure.loadStructure(load_structure_path)
structure.lattice


Lattice(a=6.3613, b=6.3613, c=6.3613, alpha=90, beta=90, gamma=90)

## Calculate the rotation for each vector
Finds the rotation angles that takes [001] to a given zone axis.

In [109]:
def get_rotation_from_z_to_direction(structure, direction):
    """
    Finds the rotation that takes [001] to a given zone axis.
    Parameters
    ----------
    structure : diffpy.structure
        The structure for which a rotation needs to be found
    direction : array like
        [UVW] direction that the 'z' axis should end up point down.
    Returns
    -------
    euler_angles : tuple
        'rzxz' in degrees
    See Also
    --------
    generate_zap_map
    get_grid_around_beam_direction
    Notes
    -----
    This implementation works with an axis arrangement that has +x as left to right,
    +y as bottom to top and +z as out of the plane of a page. Rotations are counter clockwise
    as you look from the tip of the axis towards the origin
    """

    # Case where we don't need a rotation, As axis is [0,0,z] or [0,0,0]
    if np.dot(direction, [0, 0, 1]) == np.linalg.norm(direction):
        return (0, 0, 0)

    # Normalize our directions
    cartesian_direction = structure.lattice.cartesian(direction)
    cartesian_direction = cartesian_direction / np.linalg.norm(cartesian_direction)

    # Find the rotation using cartesian vector geometry
    rotation_axis = np.cross([0, 0, 1], cartesian_direction)
    rotation_angle = np.arccos(np.dot([0, 0, 1], cartesian_direction))
    euler = axangle2euler(rotation_axis, rotation_angle, axes="rzxz")
    return np.rad2deg(euler)

In [110]:
euler_angles = []
for i in range(0, len(vectors)):
    angles=get_rotation_from_z_to_direction(structure,vectors[i])
    euler_angles.append(angles)

In [111]:
euler_angles = np.array(euler_angles)
where_nan= np.isnan(euler_angles)
euler_angles[where_nan]=0
convert = map(tuple, euler_angles)
euler_angles = tuple(convert)

In [112]:
len(euler_angles)

9

## Simulate theoretical diffraction patterns in the plausible zone axis
Set up the detector parameters.
The diffraction generator will simulate diffraction of the structure file for every plausible zone axis.

In [113]:
# From the parameters stated at the beginning

half_pattern_size = pattern_size // 2
calibration = reciprocal_radius / half_pattern_size

ediff = DiffractionGenerator(beam_energy, max_excitation_error)  # keV and relrod length (1/Å)

In [114]:
phase_names = [cif_name]
sample_lib = StructureLibrary(phase_names, [structure], [list(euler_angles)])

diff_gen = DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                          calibration =calibration,
                                          reciprocal_radius=reciprocal_radius,
                                          half_shape = (half_pattern_size, half_pattern_size),
                                          with_direct_beam = True)
data_phase=[]
for i in range(0, len(euler_angles)):
    pattern = sim_as_signal(library.get_library_entry(phase = cif_name, angle = (euler_angles[i]))['Sim'],
                       pattern_size, max_excitation_error, reciprocal_radius)
    data_phase.append(pattern)

data = [x.data for x in data_phase]
test_data = pxm.ElectronDiffraction2D(np.asarray(data).reshape(1,len(euler_angles), pattern_size, pattern_size))
test_data.set_diffraction_calibration(calibration)

## Plot results

In [115]:
# Correct the beam shift to match the simulated data center
shift = v0

peaks2D[:,0] -= shift[0]
peaks2D[:,1] -= shift[1]

#Reshape navigation axis to only 1 dim
test_data = test_data.inav[:,0]

In [116]:
spot_size = 100
test_data.plot(vmax = 0.01, cmap = 'inferno')
markers = (hs.plot.markers.point(p[0],p[1],size=spot_size, color='green') for p in peaks2D)
c_marker = (hs.plot.markers.point(v0[0], v0[1], size=spot_size, color='yellow'),
            hs.plot.markers.text(v0[0], v0[1], 'v0', color='black'),)
text_markers = (hs.plot.markers.text(v1[0], v1[1], 'v{}'.format(p1_index), color='white'),
                hs.plot.markers.point(v1[0],v1[1], size=spot_size*1.5, color='white'),
                hs.plot.markers.text(v2[0], v2[1], 'v{}'.format(p2_index), color='white'),
                hs.plot.markers.point(v2[0],v2[1], size=spot_size*1.5, color='white'))

zone_axis_markers = hs.plot.markers.text(tuple(np.full(len(vectors), -1)), tuple(np.full(len(vectors), -1)), tuple(str(s) for s in vectors), color='white')

test_data.add_marker(text_markers)
test_data.add_marker(markers)
test_data.add_marker(c_marker)
test_data.add_marker(zone_axis_markers)

# END